### Import Requirements

In [10]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from collections import Counter
import time
import pickle
from transformers import Trainer,TrainingArguments,BartConfig,BartTokenizer,BartModel,BartForConditionalGeneration,BartTokenizerFast
from torch.utils import data as data_utils
from torch.utils.data import Dataset, DataLoader

import torch

### Load Data

"big_data" contains precomputed simulations for 198 length sequences.

Do no recommend for CPU systems, or if you want to do this quickly.

In [6]:
big_data = "NP_sim_output198_1000000.txt"
big_data_header = ["dna","length","energy","struct","blank","prob"]

"small_data" is only of length 32, with fewer sequences and is more managable

Future work will be on a more flexible model that can handle variable lengths, but current intended use is only on sequences of a known set length.

In [7]:
small_data = "dna_small_sim_output.txt"
small_data_header = ["dna","length","energy","struct"]

Load prefered data set into a pandas dataframe

In [8]:
header_names = small_data_header
data = pd.read_csv(small_data,sep="\\t",header=None,names=header_names)

C:\Users\Shoshi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


Break the DNA and structure up for the tokenizer 

This is because the tokenizer expects to see words, not individual characters 

(it was made originally for NLP)

(building custom tokenizer may be possible, but model was prebuilt to be used with this tokenizer)

In [12]:
dna = data["dna"]
struct = data["struct"]

dna_list = [" ".join(list(d)) for d in data["dna"]]
struct_list = [" ".join(list(s)) for s in data["struct"]]

Split the data set into a train and validation set

In [ ]:
length = len(dna) # total number of elements

In [14]:
# run this block for an easier, shorter run
length = 10000

In [17]:
split = int(length * 0.75) # default for this is 75/25 train-val.

dna_train = dna_list[:split]
dna_val = dna_list[split:]

struct_train = struct_list[:split]
struct_val = struct_list[split:]


Using a prebuilt tokenizer that is known to work with this model

NOTE: may take a while on first run

In [4]:
tokenizer = BartTokenizerFast.from_pretrained("roberta-base")

Tokenize the training and validation dataset

In [22]:
tokenized_train = tokenizer.prepare_seq2seq_batch(src_texts = dna_train, 
                                                  tgt_texts = struct_train,
                                                  padding=True,
                                                  return_tensors='pt',
                                                  truncation=True,
                                                  #return_token_type_ids = True,
                                                  max_length=202)

In [21]:
tokenized_val = tokenizer.prepare_seq2seq_batch(src_texts = dna_val, 
                                                tgt_texts = struct_val,
                                                padding=True,
                                                return_tensors='pt',
                                                truncation=True,
                                                #return_token_type_ids = True,
                                                max_length=202)

Custom dataset class because pytorch

### Model and Training

In [23]:
class MyDataset(data_utils.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        # self.encodings.keys() = ['input_ids', 'attention_mask', 'start_positions', 'end_positions']
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

Create pyTorch Datasets for training (tds) and evalidation (eds)

In [24]:
tds = MyDataset(tokenized_train)
eds = MyDataset(tokenized_val)

Arguments for the trainer

In [30]:
training_args = TrainingArguments(
    output_dir = "./BART_Small_Output", # where to store the checkpoints
    logging_dir = "./BART_Small_log", # logging directory
    per_device_train_batch_size=64,  # batch size per device during training NOTE: if on CPU or if you get OOM errors, set to smaller number
    per_device_eval_batch_size=64,   # batch size for evaluation NOTE: if on CPU or if you get OOM errors, set to smaller number
    fp16 = True, # if having NAN errors, disable. keeping true helps run faster
    num_train_epochs=10) # number of training epochs to run

Configure and create model

In [27]:
config = BartConfig(
    d_model = 256,
    encoder_layers=6,
    decoder_layers=6,
    encoder_attention_heads=8,
    decoder_attention_heads=8,
    decoder_ffn_dim=1024,
    encoder_ffn_dim=1024
    )
model = BartForConditionalGeneration (config)

Uncommont and fill in URI for a previous training checkpoint

In [25]:
#model = BartForConditionalGeneration.from_pretrained('.\<Path To Model>', return_dict=True)

Create the trainer from previous arguments and the model

In [31]:
trainer = Trainer(model = model, 
                         args = training_args, 
                         train_dataset =tds,
                         eval_dataset = eds)

Run the trainer

In [32]:
trainer.train()

C:\Users\Shoshi\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


Step,Training Loss
500,2.489514
1000,0.364157


C:\Users\Shoshi\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
C:\Users\Shoshi\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


TrainOutput(global_step=1180, training_loss=1.261438298629502)

Save the final result

In [33]:
model.save_pretrained('DNA_BART_32')

Open the testing notebook to try out your model